# Alcohol Metabolism in Men and Women - An observational Study
Women exhibit a lower tolerance for alcohol and develop alcohol-related liver disease more readily than men.  When men and women of the same size and drinking history consume equal amounts of alcohol, the women on average carry a higher concentration of alcohol in their bloodstream.  According to a team of Italian researchers, this occurs because alcohol-degrading enzymes in the stomach (where alcohol is partially metabolized before it enters the bloodstream and is eventually metabolized by the liver) are more active in men that in women.  The researchers studies the extent to which the activity of the enzyme explained the first-pass alcohol metabolism and the extent to which ist explained the first-pass metabolism between women and men.  (Data from M. Frezza et al., "High Blood Alcohol Levels in Women," *New England Journal of Medicine* 322 (1990): 95-99.)

The subjects were 18 men and 14 women, all living in Trieste.  Three of the women and five of the men were categorized as alcoholic.  All subjects received ethanol, at a dose of 0.3 grams per kilogram of body weight, orally one day and intravenously another, in randomly determined order.  Since the intravenous administration bypasses the stomach, the difference in blood alcohol concentration - the concentration after intravenous administration minus the concentration after oral administration - provides a measure of "first-pass metabolism" in the stomach.  In addition, gastric alcohol dehydrogenase (AD) activity (activity of the key enzyme) was measured in mucus samples taken from the stomach linings.

Several questions arise.  Do levels of first-pass metabolism differ between men and women?  Can the differences be explained by postulating that men have more dehydrogenase activity in their stomachs?  Are the answers to these questions complicated by an alcoholism effect?

In [ ]:
# standard library imports

# 3rd party library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import statsmodels.formula.api as smf

plt.rcParams['text.usetex'] = True
pd.options.display.float_format = "{:.2f}".format
sns.set()

We begin by reading the data and summarizing the variables.

In [ ]:
df = pd.read_csv('case1101.csv')
df.head()

In [ ]:
df.groupby(['Sex', 'Alcohol'])[['Metabol', 'Gastric']].std()
      

In [ ]:
g = sns.relplot(data=df, x='Gastric', y='Metabol', col='Sex', hue='Alcohol', kind='scatter')
_ = g.set_xlabels(r'Gastric AD Activity ($\mu$mol/min/g of tissue)')
_ = g.set_ylabels(r'First-Pass Metabolism (mmol/liter-hour)')

In [ ]:
g = sns.catplot(x='Gastric', y='Metabol', hue='Sex', col='Alcohol', data=df)

This does not lend any wisdom one way or the other as to whether alcoholism is really a factor or not.  There are only 8 alcoholic subjects in the study.

In [ ]:
formula = (
    'Metabol ~ Gastric + Sex + Alcohol '
    '+ Gastric:Sex + Sex:Alcohol + Gastric:Alcohol '
    '+ Gastric:Sex:Alcohol'
)
model = smf.ols(formula, data=df)
model_fit = model.fit()
model_fit.summary()

In [ ]:
ax = sns.scatterplot(results.fittedvalues, results.resid)
ax.set_xlabel('Fitted Values')
ax.set_ylabel('Residuals')

In [ ]:
df[results.fittedvalues > 6]

In [ ]:
infl = model_fit.get_influence()
infldf = infl.summary_frame()
fig, axes = plt.subplots(nrows=2, ncols=1)
sns.scatterplot(df.Subject, infldf['standard_resid'], ax=axes[0])
axes[0].set_ylabel('Internally\nStudentized\nResiduals')
sns.scatterplot(df.Subject, infldf['cooks_d'], ax=axes[1])
axes[1].set_ylabel("Cook's\nDistance")
                                 

In [ ]:
infl.plot_influence()

This suggests that Subjects 31 and 32 might be disregarded.

In [ ]:
df = df.loc[df.Subject < 31, :]
df.describe()

This suggests that Subjects 31 and 32 might be disregarded.

Now we go back to refining the model.  First of all, it is not clear that alcohol is a factor at all.  We can do an extra sums of squares $F$-test for that.

In [ ]:
formula = (
    'Metabol ~ Gastric + Sex + Alcohol '
    '+ Gastric:Sex + Sex:Alcohol + Gastric:Alcohol '
    '+ Gastric:Sex:Alcohol'
)
model = smf.ols(formula, data=df)
model_fit = model.fit()
model_fit.summary()

formula_reduced = 'Metabol ~ Gastric + Sex + Gastric:Sex'
model_reduced = smf.ols(formula_reduced, data=df)
model_rfit = model_reduced.fit()
model_rfit.summary()

In [ ]:
f_stat = ((model_rfit.ssr - model_fit.ssr) / (model_rfit.df_resid - model_fit.df_resid)) / model_fit.mse_resid    
f_stat

In [ ]:
fpvalue = scipy.stats.f.sf(4, 24, f_stat)
fpvalue

This strongly suggests that all of the coefficients for an alcohol effect are zero and can be dropped from the model.  We can additionally check as to whether we can drop sex from the model.

In [ ]:
formula_reduced2 = 'Metabol ~ Gastric'
model_rfit2 = smf.ols(formula_reduced2, data=df).fit()
model_rfit2.summary()

In [ ]:
f_stat = ((model_rfit2.ssr - model_rfit.ssr) / (model_rfit2.df_resid - model_rfit.df_resid)) / model_rfit.mse_resid    
f_stat

In [ ]:
scipy.stats.f.sf(model_rfit2.df_resid - model_rfit.df_resid, model_rfit.df_resid, f_stat)  

This $p$-value is too high.  We cannot conclude that we can drop both the sex and the sex-gastric interaction term from the model.  How about just the sex term?

In [ ]:
formula_reduced3 = 'Metabol ~ Gastric + Sex:Gastric'
model_rfit3 = smf.ols(formula_reduced3, data=df).fit()
model_rfit3.summary()

In [ ]:
f_stat = ((model_rfit3.ssr - model_rfit.ssr) / (model_rfit3.df_resid - model_rfit.df_resid)) / model_rfit.mse_resid    
scipy.stats.f.sf(model_rfit3.df_resid - model_rfit.df_resid, model_rfit.df_resid, f_stat)  

That provides strong evidence that the sex term can be dropped.  And finally, it makes sense to drop the intercept as well, since there is no first-pass metabolism if there is no activity of the enzyme.

In [ ]:
formula_reduced4 = 'Metabol ~ Gastric + Sex:Gastric - 1'
model_rfit4 = smf.ols(formula_reduced4, data=df).fit()
model_rfit4.summary()

In [ ]:
f_stat = ((model_rfit4.ssr - model_rfit3.ssr) / (model_rfit4.df_resid - model_rfit3.df_resid)) / model_rfit3.mse_resid    
scipy.stats.f.sf(model_rfit4.df_resid - model_rfit3.df_resid, model_rfit3.df_resid, f_stat) 

Again, the $F$-test shows that this is justified, so here we have our final model.  

$\mu\{metabolism|gast,sex\} = \beta_1 gast + \beta_2 gast \times sex$

Note that for any level of gastric AD activity, the mean first pass metabolism for men exceeds that of women by the ratio of $\frac{\beta_1 + \beta_2}{\beta_1} = 2.203$.

# Statistical Conclusion
The following inferences pertain only to individuals with gastric AD activity levels between 0.8 and 3.0 $\mu$mol/min/g.  No reliable model could be determined for values greater than 3.0  There was no evidence from the data that alcoholism was related to first-pass metabolism in any way ($p$-value = 0.93, from an F-test for significance of alcoholism and its interaction with gastric activity and sex.)  Convincing evidence exists that first-pass metabolism was larger for males than for females overall (two-sided $p$-value = 0.0002, from a rank-sum test) and that gastric AD activity was larger for males than for females (two-sided $p$-value = 0.07 from a rank-sum test).  Males had higher first-pass metabolism than females even accounting for differences in gastric AD activity (two-sided $p$-value = 0.0003 from a $t$-test for equality of male and female slopes when both intercepts are zero).  For a given level of gastric dehydrogenase activity, the mean first-pass alcohol metabolism for men is estimated to be 2.20 times as large as the first-pass alcohol metabolism for women (approximate 95% confidence interval from 1.37 to 3.04).